In [21]:
from data.m2or import M2OR_Pairs
from dgllife.utils import SMILESToBigraph
from dgllife.utils import CanonicalAtomFeaturizer
from dgllife.utils import SMILESToBigraph


smiles_to_g = SMILESToBigraph(add_self_loop=True, node_featurizer=CanonicalAtomFeaturizer(),
                                edge_featurizer=None)    


dataset = M2OR_Pairs(smiles_to_graph=smiles_to_g, 
        n_jobs=1, cross_attention=True, load_full = True)

loading esm model...
done loading esm model


KeyboardInterrupt: 

In [ ]:
dataset.seq_embeddings_dict[dataset.seq_id[3343]].shape

(705, 1280)

In [ ]:
dataset.graph_mask[3343]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [ ]:
data_path = 'data/datasets/M2OR_original_mol_OR_pairs.csv'
import pandas as pd

df = pd.read_csv(data_path, sep = ';')
df

,mol_id,seq_id,Responsive,_DataQuality,num_unique_value_screen,canonicalSMILES,mutated_Sequence
0,m_0,s_29,1,ec50,NaN,CC1=CCC(CC1)C(=C)C,MRENNQSSTLEFILLGVTGQQEQEDFFYILFLFIYPITLIGNLLIV...
1,m_0,s_449,1,ec50,NaN,CC1=CCC(CC1)C(=C)C,MDQSNYSSLHGFILLGFSNHPKMEMILSGVVAIFYLITLVGNTAII...
2,m_0,s_539,0,ec50,NaN,CC1=CCC(CC1)C(=C)C,MDQSNYSSLHGFILLGFSNHPKMEMILSGVVAIFYLITLVGNTAII...
3,m_1,s_1,1,ec50,NaN,CC1=CCC(CC1=O)C(=C)C,MTEDNYSLTTEFILIGFSDHPDLKILLFLVLSTIYLVTMVGNLGLV...
4,m_1,s_1051,0,ec50,NaN,CC1=CCC(CC1=O)C(=C)C,MREENESSTTDFTLLGVTRQREQEYFFFILFLFIYPITVFGNMLII...
...,...,...,...,...,...,...,...
46558,m_99,s_443,1,secondaryScreening,4.0,C1=CC=C(C=C1)OC2=CC=CC=C2,MRENNQSSTLEFILLGVTGQQEQEDFFYILFLFIYPITLIGNLLIV...
46559,m_99,s_444,0,secondaryScreening,4.0,C1=CC=C(C=C1)OC2=CC=CC=C2,MRENNQSSTLEFILLGVTGQQEQEDFFYILFLFIYPITLIGNLLIV...
46560,m_99,s_449,0,secondaryScreening,4.0,C1=CC=C(C=C1)OC2=CC=CC=C2,MDQSNYSSLHGFILLGFSNHPKMEMILSGVVAIFYLITLVGNTAII...
46561,m_99,s_450,0,secondaryScreening,4.0,C1=CC=C(C=C1)OC2=CC=CC=C2,MDEANHSVVSEFVFLGLSDSRKIQLLLFLFFSVFYVSSLMGNLLIV...


In [ ]:
max(df['mutated_Sequence'].tolist(), key=len)

'MQPTATMATAATTTTTTTATVALTTSWDNATGRPTAEPDPILDNYVLLVVVMSLFVGGTLVVLSGVLLLCKRCWDVHQRLNRAMEEAEKTTTTYLDNGTHPAQDPDFRGEDPECQDAETERFLSTSSTGRRVSFNEAALFEQSRKTQDKGRRYTLTEGDFHHLKNARLTHLHLPPLKIVTIHECDSGEASSATTPHPATSPKATLAIFQPPGKALTGRSVGPSSALPGDPYNSAAGATDFAEISPSASSDSGEGTSLDAGTRSTKAGGPGAAAGPGEAGPGSGAGTVLQFLTRLRRHASLDGASPYFKVKKWKLEPSQRAASLDTRGSPKRHHFQRQRAASESTEQEEGDAPQEDFIQYIARAGDAVAFPHPRPFLASPPPALGRLEAAEAAGGASPDSPPERGAGSAGPEQQQPPLEPDAERDAGPEQAQTSYRDLWSLRASLELHAAASDHSSSGNDRDSVRSGDSSGSGSGGAAPAFPPPSPPAPRPKDGEARRLLQMDSGYASIEGRGAGDDTEPPAAPARPRSPRAWPRRPRRDYSIDEKTDALFHEFLRHDPHFDDTPAAARHRARAHPHARKQWQRGRQHSDPGARAAPALAGTPAPPAGAARPARAPLRRGDSVDGPPDGRTLGGAGDDPAIPVIEEEPGGGGCPGSGLCVLPSGSVLDKLAAGLDERLFPPRLAEPVVATPALVAAAPTSPDHSPA'

In [ ]:
len('MQPTATMATAATTTTTTTATVALTTSWDNATGRPTAEPDPILDNYVLLVVVMSLFVGGTLVVLSGVLLLCKRCWDVHQRLNRAMEEAEKTTTTYLDNGTHPAQDPDFRGEDPECQDAETERFLSTSSTGRRVSFNEAALFEQSRKTQDKGRRYTLTEGDFHHLKNARLTHLHLPPLKIVTIHECDSGEASSATTPHPATSPKATLAIFQPPGKALTGRSVGPSSALPGDPYNSAAGATDFAEISPSASSDSGEGTSLDAGTRSTKAGGPGAAAGPGEAGPGSGAGTVLQFLTRLRRHASLDGASPYFKVKKWKLEPSQRAASLDTRGSPKRHHFQRQRAASESTEQEEGDAPQEDFIQYIARAGDAVAFPHPRPFLASPPPALGRLEAAEAAGGASPDSPPERGAGSAGPEQQQPPLEPDAERDAGPEQAQTSYRDLWSLRASLELHAAASDHSSSGNDRDSVRSGDSSGSGSGGAAPAFPPPSPPAPRPKDGEARRLLQMDSGYASIEGRGAGDDTEPPAAPARPRSPRAWPRRPRRDYSIDEKTDALFHEFLRHDPHFDDTPAAARHRARAHPHARKQWQRGRQHSDPGARAAPALAGTPAPPAGAARPARAPLRRGDSVDGPPDGRTLGGAGDDPAIPVIEEEPGGGGCPGSGLCVLPSGSVLDKLAAGLDERLFPPRLAEPVVATPALVAAAPTSPDHSPA')

705

In [ ]:
import torch
esm_model = None
esm_alphabet = None
def setup_esm(device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')):
    import esm
    # Load ESM-2 model
    global esm_model
    global esm_alphabet
    if esm_model is None:
        print('loading esm model...')
        esm_model, esm_alphabet = esm.pretrained.esm2_t33_650M_UR50D() ######################################
        esm_model.eval()  # disables dropout for deterministic results
        esm_model.to(device)
        print('done loading esm model')
    return esm_model, esm_alphabet

def esm_embed(sequences, device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'), per_residue = False):
    # get the embeddings for a list of sequences. Code is copied from ESM github readme
    assert isinstance(sequences, list)
    esm_model, esm_alphabet = setup_esm()
    batch_converter = esm_alphabet.get_batch_converter()
    
    def divide_chunks(l, n):
        for i in range(0, len(l), n):
            yield l[i:i + n]

    sequence_representations = []
    sequence_chunks = list(divide_chunks(sequences, 5))
    for sequence_chunk in sequence_chunks:
        data = []
        for i, sequence in enumerate(sequence_chunk):
            assert ' ' not in sequence
            if len(sequence) > 1600:
                print('trimming sequence to 1600 amino acids max')
                sequence = sequence[0:1600]
            data.append(('protein' + str(i), sequence))
    
        batch_labels, batch_strs, batch_tokens = batch_converter(data)
        batch_lens = (batch_tokens != esm_alphabet.padding_idx).sum(1)

        # Extract per-residue representations (on CPU)
        with torch.inference_mode():
            results = esm_model(batch_tokens.to(device), repr_layers=[33], return_contacts=False)
        token_representations = results["representations"][33]
        #print(token_representations.shape)

        # Generate per-sequence representations via averaging
        # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
        #print (batch_lens)
        if per_residue:
            for i, tokens_len in enumerate(batch_lens):
                sequence_representations.append(token_representations[i, 1 : -1].detach().cpu().numpy())
        else:
            for i, tokens_len in enumerate(batch_lens):
                sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0).detach().cpu().numpy())
    return sequence_representations



In [ ]:
esm_embed(['AAA<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'], per_residue=True)[0].shape

(283, 1280)

In [ ]:
import torch.nn as nn

class CrossAttention(nn.Module):
    
    """Cross-Attention Block of ligand-protein model, that takes in two 2d tensors for the molecular and protein embeddings, collapses them to the same
    dimension, and performs a cross-attention between them. 
    
    Args:
        D1 (int): dimension of input protein tensor
        D2 (int): dimension of input mol tensor (usually smaller)
    """

    def __init__(self, D1, D2):
        super(CrossAttention, self).__init__()

        # Define the trainable weight matrices for query, key, and value transformations
        self.query_transform_tensor1 = nn.Linear(D1, D2)
        self.key_transform_tensor1 = nn.Linear(D1, D2)
        self.value_transform_tensor1 = nn.Linear(D1, D2)

        self.query_transform_tensor2 = nn.Linear(D2, D2)
        self.key_transform_tensor2 = nn.Linear(D2, D2)
        self.value_transform_tensor2 = nn.Linear(D2, D2)

        # 1D Convolutional layer for aggregation
        self.linear1 = nn.Linear(D2, 1)
        self.linear2 = nn.Linear(D2, 1)

    def scaled_dot_product_attention(self, query, key, value):
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_k).float())
        #attention_weights = torch.softmax(scores, dim=-1) ## try relu here, then softmax at the end w/ the MLP prediction head
        attention_weights = torch.relu(scores)
        attended_values = torch.matmul(attention_weights, value)
        return attended_values

    def forward(self, tensor1, tensor2):
        # Compute query, key, and value representations for both tensors
        query_tensor1 = self.query_transform_tensor1(tensor1)
        key_tensor1 = self.key_transform_tensor1(tensor1)
        value_tensor1 = self.value_transform_tensor1(tensor1)

        query_tensor2 = self.query_transform_tensor2(tensor2)
        key_tensor2 = self.key_transform_tensor2(tensor2)
        value_tensor2 = self.value_transform_tensor2(tensor2)

        # Compute cross-attention between tensor1 and tensor2
        attended_values_tensor1 = self.scaled_dot_product_attention(query_tensor1, key_tensor2, value_tensor2) ## outputs (batch_size, R, D2)
        attended_values_tensor2 = self.scaled_dot_product_attention(query_tensor2, key_tensor1, value_tensor1) ## outputs (batch_size, A, D2)
        print(attended_values_tensor1.shape)
        print(attended_values_tensor2.shape)
        #attended_values_tensor2 = self.scaled_dot_product_attention(query_tensor2, key_tensor1, value_tensor1)

        # Apply Linear for aggregation
        fixed_size_tensor1 = self.linear1(attended_values_tensor1).squeeze(-1) ## outputs (batch_size, R)
        fixed_size_tensor2 = self.linear2(attended_values_tensor2).squeeze(-1) ## outputs (batch_size, R)
        print(fixed_size_tensor1.shape)
        print(fixed_size_tensor2.shape)
        return fixed_size_tensor1, fixed_size_tensor2

In [ ]:
# Example usage:
R, D1 = 5, 64
A, D2 = 7, 32

tensor1 = torch.randn(R, D1)
tensor2 = torch.randn(A, D2)

cross_attention = CrossAttention(D1, D2)
output1, output2 = cross_attention(tensor1, tensor2)

print("Output1 shape:", output1.shape)  # Output1 shape: torch.Size([1, 32])
print("Output2 shape:", output2.shape)  # Output2 shape: torch.Size([1, 32])
#In this example, output1 and output2 are the fixed-size vector representations of the two tensors after cross-attention, with shape (1, D2).



torch.Size([5, 32])
torch.Size([7, 32])
torch.Size([5])
torch.Size([7])
Output1 shape: torch.Size([5])
Output2 shape: torch.Size([7])


In [ ]:
from dgllife.utils import ScaffoldSplitter, RandomSplitter

train_set, val_set, test_set = RandomSplitter.train_val_test_split(
    dataset, frac_train=0.8, frac_val=0.1, frac_test=0.1, random_state = 42)



In [ ]:
import dgl 
import torch
def collate_molgraphs(data):
    """Batching a list of datapoints for dataloader.

    Parameters
    ----------
    data : list of 3-tuples or 4-tuples.
        Each tuple is for a single datapoint, consisting of
        a SMILES, a DGLGraph, all-task labels and optionally a binary
        mask indicating the existence of labels.

    Returns
    -------
    smiles : list
        List of smiles
    bg : DGLGraph
        The batched DGLGraph.
    labels : Tensor of dtype float32 and shape (B, T)
        Batched datapoint labels. B is len(data) and
        T is the number of total tasks.
    masks : Tensor of dtype float32 and shape (B, T)
        Batched datapoint binary mask, indicating the
        existence of labels.
    """
    ids, seq_ids, sequences_dict, seq_embeddings = None, None, None, None
    if len(data[0]) == 3:
        smiles, graphs, labels = map(list, zip(*data))
    elif len(data[0]) == 9:
        smiles, graphs, labels, masks, ids, seq_ids, sequences_dict, seq_embeddings, sample_weights = map(list, zip(*data))
    elif len(data[0]) == 11:
        smiles, graphs, labels, masks, ids, seq_ids, sequences_dict, seq_embeddings, sample_weights, seq_mask, node_mask = map(list, zip(*data))
    else:
        smiles, graphs, labels, masks = map(list, zip(*data))

    bg = dgl.batch(graphs)
    bg.set_n_initializer(dgl.init.zero_initializer)
    bg.set_e_initializer(dgl.init.zero_initializer)
    labels = torch.stack(labels, dim=0)

    if len(data[0]) == 3:
        masks = torch.ones(labels.shape)
    else:
        masks = torch.stack(masks, dim=0)
    
    if len(data[0]) == 9:
        return smiles, bg, labels, masks, ids, seq_ids, sequences_dict, seq_embeddings, sample_weights
    elif len(data[0]) == 11:
        return smiles, bg, labels, masks, ids, seq_ids, sequences_dict, seq_embeddings, sample_weights, seq_mask, node_mask
    return smiles, bg, labels, masks


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=True,
                            collate_fn=collate_molgraphs, num_workers=1)



In [ ]:
import numpy as np 
for batch_id, batch_data in enumerate(train_loader):
    smiles, bg, labels, masks, ids, seq_ids, sequences_dict, seq_embeddings, sample_weights, seq_mask, node_mask = batch_data
    
    seq_emb_arr = np.dstack(seq_embeddings)
    seq_embeddings_tensor = torch.FloatTensor(np.rollaxis(seq_emb_arr, -1)).cuda()
    print(seq_embeddings_tensor.shape)
    #print(seq_embeddings.shape)
    seq_mask = np.vstack(seq_mask)
    seq_mask = torch.FloatTensor(seq_mask).cuda()
    
    node_mask = np.vstack(node_mask)
    node_mask = torch.FloatTensor(node_mask).cuda()

    print(seq_mask.shape, node_mask.shape)
    
    

torch.Size([64, 705, 1280])
torch.Size([64, 705]) torch.Size([64, 22])
torch.Size([64, 705, 1280])
torch.Size([64, 705]) torch.Size([64, 22])
torch.Size([64, 705, 1280])
torch.Size([64, 705]) torch.Size([64, 22])
torch.Size([64, 705, 1280])
torch.Size([64, 705]) torch.Size([64, 22])
torch.Size([64, 705, 1280])
torch.Size([64, 705]) torch.Size([64, 22])
torch.Size([64, 705, 1280])
torch.Size([64, 705]) torch.Size([64, 22])
torch.Size([64, 705, 1280])
torch.Size([64, 705]) torch.Size([64, 22])


KeyboardInterrupt: 

In [ ]:
node_mask[0].shape

torch.Size([22])

In [ ]:
max_seq_len = len(max(df['mutated_Sequence'].tolist(), key=len))

In [ ]:
## Create a 2d tensor of shape (len(df), max_seq_len), and for each row i, fill it with an array of len(sequences[i]) 1's and the rest are 0's. 
## This will be used as the attention mask for the cross attention layer.
sequences = df['mutated_Sequence'].tolist()
seq_mask = torch.zeros((len(df), max_seq_len))
for i in range(len(df)):
    seq_mask[i, :len(sequences[i])] = 1

In [ ]:
seq_mask.shape

torch.Size([46563, 705])

In [ ]:
dataset.graphs[0].num_nodes()

10

In [ ]:
max_node_len = max([graph.num_nodes() for graph in dataset.graphs])

In [ ]:
seq_mask[44384].shape

torch.Size([705])

In [ ]:
graph_mask = torch.zeros((len(dataset.graphs), max_node_len))
for idx in range(len(dataset)):
    graph_mask[idx, :dataset.graphs[idx].num_nodes()] = 1

In [ ]:
graph_mask[4444]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [ ]:
len(dataset.graphs)

46563

In [ ]:
graph_mask[1:434].shape

torch.Size([433, 22])

In [ ]:
import torch.nn as nn
class CrossAttention(nn.Module):
    
    """Cross-Attention Block of ligand-protein model, that takes in two 2d tensors for the molecular and protein embeddings, collapses them to the same
    dimension, and performs a cross-attention between them. 
    
    Args:
        D1 (int): dimension of input protein tensor
        D2 (int): dimension of input mol tensor (usually smaller)
    """

    def __init__(self, D1, D2):
        super(CrossAttention, self).__init__()

        # Define the trainable weight matrices for query, key, and value transformations
        self.query_transform_tensor1 = nn.Linear(D1, D2)
        self.key_transform_tensor1 = nn.Linear(D1, D2)
        self.value_transform_tensor1 = nn.Linear(D1, D2)

        self.query_transform_tensor2 = nn.Linear(D2, D2)
        self.key_transform_tensor2 = nn.Linear(D2, D2)
        self.value_transform_tensor2 = nn.Linear(D2, D2)

        # Linear layer for aggregation
        self.linear1 = nn.Linear(D2, 1)
        self.linear2 = nn.Linear(D2, 1)

    def scaled_dot_product_attention(self, query, key, value):
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_k).float())
        #attention_weights = torch.softmax(scores, dim=-1) ## try relu here, then softmax at the end w/ the MLP prediction head
        attention_weights = torch.relu(scores)
        attended_values = torch.matmul(attention_weights, value)
        return attended_values

    def forward(self, tensor1, tensor2, seq_mask = None, node_mask = None):
        # Compute query, key, and value representations for both tensors
        tensor1 = tensor1 * seq_mask[:, :, np.newaxis]
        query_tensor1 = self.query_transform_tensor1(tensor1)
        key_tensor1 = self.key_transform_tensor1(tensor1)
        value_tensor1 = self.value_transform_tensor1(tensor1)

        query_tensor2 = self.query_transform_tensor2(tensor2)
        key_tensor2 = self.key_transform_tensor2(tensor2)
        value_tensor2 = self.value_transform_tensor2(tensor2)

        # Compute cross-attention between tensor1 and tensor2
        attended_values_tensor1 = self.scaled_dot_product_attention(query_tensor1, key_tensor2, value_tensor2) ## outputs (batch_size, R, D2)
        attended_values_tensor2 = self.scaled_dot_product_attention(query_tensor2, key_tensor1, value_tensor1) ## outputs (batch_size, A, D2)
        print(attended_values_tensor1.shape)
        print(attended_values_tensor2.shape)
        #attended_values_tensor2 = self.scaled_dot_product_attention(query_tensor2, key_tensor1, value_tensor1)

        # Apply Linear for aggregation
        fixed_size_tensor1 = self.linear1(attended_values_tensor1).squeeze(-1) ## outputs (batch_size, R)
        fixed_size_tensor2 = self.linear2(attended_values_tensor2).squeeze(-1) ## outputs (batch_size, R)
        
        print(fixed_size_tensor1.shape, fixed_size_tensor2.shape)
        
        return fixed_size_tensor1, fixed_size_tensor2    


In [ ]:
R, D1 = 705, 1280
A, D2 = 22, 256

tensor1 = torch.randn(2, R, D1)
tensor2 = torch.randn(2, A, D2)

mask1 = torch.ones((2,R))
mask1[400:] = 0


In [ ]:
(tensor1 * mask1[:, :, np.newaxis]).shape


torch.Size([2, 705, 1280])

In [ ]:
R, D1 = 705, 1280
A, D2 = 22, 256

tensor1 = torch.randn(2, R, D1)
tensor2 = torch.randn(2, A, D2)

cross_attention = CrossAttention(D1, D2)

seq_mask = torch.ones((2, R))

output1, output2 = cross_attention(tensor1, tensor2, seq_mask)

print("Output1 shape:", output1.shape)  # Output1 shape: torch.Size([1, 32])
print("Output2 shape:", output2.shape)  # Output2 shape: torch.Size([1, 32])
#In this example, output1 and output2 are the fixed-size vector representations of the two tensors after cross-attention, with shape (1, D2).



torch.Size([2, 705, 256])
torch.Size([2, 22, 256])
torch.Size([2, 705]) torch.Size([2, 22])
Output1 shape: torch.Size([2, 705])
Output2 shape: torch.Size([2, 22])


In [ ]:
output2

tensor([[-0.9199,  2.0333, -0.4808, -0.6681, -1.9653,  3.0707,  0.5324, -2.7096,
         -1.8324, -1.0475,  0.1393, -1.8973,  0.4469, -1.5381, -1.1548, -0.2025,
         -0.3511,  0.0686, -2.7342, -0.5233, -2.0736, -4.1108],
        [-4.1297, -2.6616,  2.1760, -1.1716,  0.5314,  1.4921,  0.1804, -1.3151,
          2.7273, -0.6498,  6.7817, -3.5639, -0.0154,  2.2489, -3.4568, -0.6560,
         -1.0690, -4.2612, -3.5600, -1.5677,  0.5680,  0.3355]],
       grad_fn=<SqueezeBackward1>)

In [ ]:
node_mask = torch.zeros((2, 22))
output2[node_mask == 0] = -np.inf

In [ ]:
output2

tensor([[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]],
       grad_fn=<AsStridedBackward0>)

Bad pipe message: %s [b'\x89c\x90n\xda\xad\xf2\x8f\xec\xc9~\xd0\x1dJ\xff\xb7\xabN \xb33qZ\xc2AQ\x16\x84I\x14\x83\xde\x8a\xdb\xff\xc8\xb2\x17#|O\xb5\xb1\xcb\x18\x90\x8a\xc5\x8f\x85\xd0\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0']
Bad pipe message: %s [b'.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04']
Bad pipe message: %s [b'\x03\x06', b'\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b'\x08\x06\x04\x01\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xaaE\x06\xb2+\xd1\xad\xff^\xc7! \xf8+\xf4\xdf\x84\x1d\xf7\xb5\xa6\x14']
Bad pipe message: %s [b'P\xcb1\xff\xd3K_z\x17O\x92\xc7\xc8\xa9\x1a\xae\xa0{ ']
Bad pipe message: %s [b'\xc8N\xcf\x84Dw\xc99\xe1\xdf\xeaM\xe3\xf5\xfb?m>\xa8/k[\x08\x03\xb1;X\xd6\xedd\x00\x08\x13\x02\x13\x03\

In [ ]:
dataset.graphs[3763].ndata['h'].shape

torch.Size([11, 74])

In [ ]:
cross_att = CrossAttention()

In [ ]:
dataset.graphs[0].ndata['h'].shape

torch.Size([10, 74])

In [ ]:
dataset.graphs[0].local_scope()

In [ ]:
import dgl
bg = dgl.batch(dataset.graphs[0:32])

In [ ]:
import torch
bg.ndata['logits'] = torch.randn(349, 256)

In [ ]:
graphs = dgl.unbatch(bg)

In [ ]:
bg

Graph(num_nodes=349, num_edges=1047,
      ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32), 'logits': Scheme(shape=(256,), dtype=torch.float32)}
      edata_schemes={})

In [ ]:
empty_batch_feats = torch.zeros((32, 22, 256))

In [ ]:
graphs[0].ndata['logits'].shape

torch.Size([10, 256])

In [ ]:
for i in range(len(graphs)):
    empty_batch_feats[i][:graphs[i].num_nodes()] = graphs[i].ndata['logits']

In [ ]:
empty_batch_feats.shape[0]

32

In [ ]:
node_feats_lst = [g.ndata['logits'] for g in graphs]

In [ ]:
torch.cat([node_feats_lst], dim=0).shape

TypeError: expected Tensor as element 0 in argument 0, but got list

In [ ]:
import numpy as np

seq_emb_arr = np.dstack(node_feats_lst)
seq_embeddings_tensor = torch.FloatTensor(np.rollaxis(seq_emb_arr, -1)).cuda()


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 10 and the array at index 3 has size 11

### Re-initialize ESM weights

In [26]:
import esm
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('loading esm model...')
esm_model, esm_alphabet = esm.pretrained.esm2_t33_650M_UR50D() ######################################
esm_model.eval()  # disables dropout for deterministic results
esm_model.to(device)
print('done loading esm model')


loading esm model...
done loading esm model


In [28]:
esm_model.layers[1].self_attn.k_proj.weight

Parameter containing:
tensor([[-0.1746,  0.0135,  0.1160,  ...,  0.0136, -0.0836,  0.0016],
        [ 0.0905, -0.0453,  0.0685,  ...,  0.0663,  0.0138,  0.0146],
        [-0.0400,  0.0616,  0.0203,  ...,  0.0631, -0.0950,  0.0014],
        ...,
        [ 0.0133, -0.0549, -0.0352,  ...,  0.0392, -0.0723, -0.1175],
        [-0.0209,  0.0558,  0.0616,  ..., -0.2120,  0.0193, -0.0345],
        [ 0.0592,  0.0552, -0.0665,  ..., -0.0968, -0.0383, -0.0413]],
       device='cuda:0', requires_grad=True)

In [29]:
import torch.nn.init as init

def reinitialize_weights(model):
    seed = 42
    torch.manual_seed(seed)
    for name, param in model.named_parameters():
        if 'embed' in name:  # Re-initialize embeddings
            if len(param.size()) > 1:
                init.normal_(param.data, mean=0.0, std=0.02)
        elif 'weight' in name:  # Re-initialize weights of linear layers
            if len(param.size()) > 1:
                init.xavier_normal_(param.data)
        elif 'bias' in name:  # Re-initialize biases of linear layers
            init.constant_(param.data, 0.0)


In [30]:
reinitialize_weights(esm_model)


In [31]:
esm_model.layers[1].self_attn.k_proj.weight

Parameter containing:
tensor([[-0.0099,  0.0066, -0.0063,  ...,  0.0732,  0.0589,  0.0390],
        [-0.0147,  0.0116, -0.0337,  ...,  0.0113,  0.0162, -0.0036],
        [-0.0050, -0.0034, -0.0055,  ...,  0.0067, -0.0099,  0.0064],
        ...,
        [ 0.0121,  0.0537,  0.0081,  ..., -0.0064,  0.0105,  0.0033],
        [ 0.0232,  0.0046, -0.0282,  ...,  0.0191,  0.0042,  0.0178],
        [-0.0004,  0.0194,  0.0183,  ..., -0.0050, -0.0182, -0.0219]],
       device='cuda:0', requires_grad=True)